In [14]:
# import the neccessary lib
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KT3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KT3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import os 
import string 
import copy 
import pickle 

title = "alt.atheism"
os.chdir("C:/Users/KT3/Desktop/20_newsgroups/20_newsgroups")
paths = [] 
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
 for i in filenames:  
    paths.append(str(dirpath)+str("\\")+i)

print('Real Path: '+dirpath)
print('My first file: '+paths[0])


Real Path: C:\Users\KT3\Desktop\20_newsgroups\20_newsgroups/alt.atheism/
My first file: C:\Users\KT3\Desktop\20_newsgroups\20_newsgroups/alt.atheism/\49960


In [16]:
# Funcations Area
def remove_stop_words(data): 
 stop_words = stopwords.words('english') 
 words = word_tokenize(str(data)) 
 new_text = "" 
 for w in words: 
    if w not in stop_words: 
        new_text = new_text + " " + w 
 return np.char.strip(new_text) 
def remove_punctuation(data): 
 symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n" 
 for i in range(len(symbols)): 
    data = np.char.replace(data, symbols[i], ' ') 
    data = np.char.replace(data, " ", " ") 
 data = np.char.replace(data, ',', '') 
 return data 
def convert_lower_case(data): 
 return np.char.lower(data) 
def stemming(data): 
 stemmer= PorterStemmer() 
 tokens = word_tokenize(str(data)) 
 new_text = "" 
 for w in tokens: 
   new_text = new_text + " " + stemmer.stem(w) 
 return np.char.strip(new_text)
def convert_numbers(data): 
 data = np.char.replace(data, "0", " zero ") 
 data = np.char.replace(data, "1", " one ") 
 data = np.char.replace(data, "2", " two ") 
 data = np.char.replace(data, "3", " three ") 
 data = np.char.replace(data, "4", " four ") 
 data = np.char.replace(data, "5", " five ") 
 data = np.char.replace(data, "6", " six ") 
 data = np.char.replace(data, "7", " seven ") 
 data = np.char.replace(data, "8", " eight ") 
 data = np.char.replace(data, "9", " nine ") 
 return data 
def remove_header(data): 
 try: 
   ind = data.index('\n\n') 
   data = data[ind:] 
 except: 
   print("No Header") 
 return data 
def remove_apostrophe(data): 
 return np.char.replace(data, "'", "")
def remove_single_characters(data): 
 words = word_tokenize(str(data)) 
 new_text = "" 
 for w in words: 
   if len(w) > 1: 
      new_text = new_text + " " + w 
 return np.char.strip(new_text) 


In [ ]:
#Exercise 1 
def preprocess(data, query):
    if not query:
        data = remove_header(data)
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = stemming(data)
    return data

In [23]:
doc = 0
postings = pd.DataFrame()

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)

    if doc % 100 == 0:
        print(doc)


    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            p.add(doc)
            postings[token][0] = p
        else:
            postings.insert(value=[{doc}], loc=0,column=token)
    doc += 1

0
100
200
300
400
500
600
700
800
No Header
900


In [25]:
postings

,kneel,hinduism,revert,infur,amok,trembl,portray,ingersol,mencken,voltair,...,nine,decemb,one,modifi,last,alt,resourc,atheism,name,archiv
0,{999},{999},{999},{999},{999},{999},{999},{999},{999},{999},...,"{0, 1, 2, 4, 5, 17, 18, 19, 22, 24, 27, 29, 30...","{0, 656, 640}","{0, 1, 2, 3, 4, 5, 7, 8, 13, 14, 16, 17, 18, 1...","{0, 1, 414, 867, 900, 901, 713, 490, 203, 204,...","{0, 1, 772, 900, 901, 263, 264, 391, 647, 145,...","{0, 1, 129, 130, 900, 901, 134, 135, 137, 139,...","{0, 98, 900, 901, 203, 205, 847, 976, 821, 662...","{0, 1, 5, 6, 517, 532, 538, 29, 542, 36, 37, 4...","{0, 1, 512, 7, 524, 34, 36, 555, 559, 561, 50,...","{0, 1, 900, 901, 203, 204, 205, 670}"


In [33]:
postings.to_pickle(title + "_unigram_postings")
postings = pd.read_pickle(title + "_unigram_postings")

In [43]:
def get_word_posting(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequancy: ",len(postings[preprocessed_word][0]))
    print("Postongs List: ",postings[preprocessed_word][0])
    
get_word_posting("nine")

nine
Frequancy:  526
Postongs List:  {0, 1, 2, 4, 5, 17, 18, 19, 22, 24, 27, 29, 30, 31, 34, 36, 37, 40, 42, 44, 46, 47, 48, 53, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 96, 99, 100, 102, 103, 105, 108, 111, 112, 113, 121, 122, 126, 134, 135, 137, 139, 140, 141, 142, 143, 145, 146, 154, 158, 160, 161, 162, 163, 164, 167, 168, 169, 170, 174, 175, 177, 178, 179, 180, 181, 183, 186, 196, 200, 201, 202, 203, 204, 205, 206, 208, 213, 215, 217, 218, 222, 224, 225, 228, 229, 230, 233, 234, 237, 239, 240, 241, 242, 244, 247, 250, 255, 259, 262, 263, 264, 265, 268, 269, 270, 271, 273, 275, 276, 277, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 299, 301, 302, 306, 315, 316, 317, 319, 320, 321, 322, 327, 328, 329, 333, 334, 335, 336, 348, 350, 354, 356, 357, 358, 359, 363, 364, 365, 370, 374, 378, 382, 383, 384, 385, 387, 388, 391, 392, 393, 394, 397, 398, 399, 400, 402, 403, 404, 405, 406, 408, 410, 411, 

In [29]:
def get_not(word):
    a = postings[word][0]
    b = set(range(len(paths)))
    return b.difference(a)

get_not("nine")

{3,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 20,
 21,
 23,
 25,
 26,
 28,
 32,
 33,
 35,
 38,
 39,
 41,
 43,
 45,
 49,
 50,
 51,
 52,
 54,
 55,
 60,
 61,
 69,
 71,
 76,
 77,
 88,
 89,
 97,
 98,
 101,
 104,
 106,
 107,
 109,
 110,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 123,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 136,
 138,
 144,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 155,
 156,
 157,
 159,
 165,
 166,
 171,
 172,
 173,
 176,
 182,
 184,
 185,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 207,
 209,
 210,
 211,
 212,
 214,
 216,
 219,
 220,
 221,
 223,
 226,
 227,
 231,
 232,
 235,
 236,
 238,
 243,
 245,
 246,
 248,
 249,
 251,
 252,
 253,
 254,
 256,
 257,
 258,
 260,
 261,
 266,
 267,
 272,
 274,
 278,
 279,
 280,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 300,
 303,
 304,
 305,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 318,
 323,
 324,
 325,
 326,
 330,
 331,
 332,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 34

In [63]:
def  generate_command_tokens(query):
    query = query.lower()
    tokens = word_tokenize(query)
    
    commands = []
    query_words = []
    
    for t in tokens:
        if t not in ['and', 'or', 'not']:
            processed_word = preprocess([t], True)
            print(str(processed_word))
            query_words.append(str(processed_word))
        else:
            commands.append(t)
            
    return commands, query_words

In [65]:
def gen_not_tuple(query_words, commands):
    tup = []
    while 'not' in commands:
        i = commands.index('not')
        word = query_words[i]
        word_postings = get_not(word)
        tup.append(word_postings)
        commands.pop(i)
        query_words[i] = i
        print('\nAfter Not Processing:', commands, query_words)
    return tup
def binary_operations(query_words, commands, tup):
    a = postings[query_words[0]][0]
    query_words.pop(0)
    for i in range(len(commands)):
        if type(query_words[i]) == int:
            b = tup.pop(0)
        else:
            b = postings[query_words[i]][0]
        if commands[i] == 'and':
            a = a.intersection(b)
        elif commands[i] == 'or':
            a = a.union(b)
        else:
            print('Invalid Command')
    return a

def execute_query(query):
    commands, query_words = generate_command_tokens(query)
    tup = gen_not_tuple(query_words, commands)
    print('\nCommands:', commands)
    print('\nQuery Words:', query_words)
    print('\nTup:', len(tup))
    final_set = binary_operations(query_words, commands, tup)
    print('\nFinal Set:', final_set)
    return final_set


def print_file(file):
    out_file = open(paths[file], 'r', encoding='cp1250')
    out_text = out_file.read()
    print(out_text)

In [ ]:
query = "exam  and resourc"
lists = execute_query(query)